In [ ]:
from pyspark.sql import SparkSession


In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
spark

Import the required dataset and transform it into a Spark dataframe

In [ ]:
data_path = '/home/danims/Documents'

In [ ]:
file_path = data_path + "/acs2017_census_tract_data.csv"
df1 = spark.read.format("csv").option("header","true").load(file_path)
# can add inferSchema=True to avoid loading as trings


Total number of columns and rows for the dataset:

In [ ]:
print(len(df1.columns))

In [ ]:
df1.count()

Importing some of the libraries and functions needed:

In [ ]:
import numpy as np
from pyspark.sql import Row
from pyspark.sql import column
from pyspark.sql.functions import lit
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.types import FloatType

Dropping variables not required for the analysis:

In [ ]:
from functools import reduce
from pyspark.sql import DataFrame

In [ ]:
df2 = reduce(DataFrame.drop, ['MeanCommute','Walk','Transit','VotingAgeCitizen',\
                              'Income','IncomeErr','IncomePerCap','IncomePerCapErr',\
                             'Professional','Service','Office','Construction',\
                              'Production','Drive','Carpool','OtherTransp','WorkAtHome',\
                             'Employed', 'PrivateWork','PublicWork',\
                              'SelfEmployed','FamilyWork', 'Unemployment'], df1)

New dataframe column names and types:

In [ ]:
df2.dtypes

Counting the number of null values in each column:

In [ ]:
from pyspark.sql.functions import isnan, when, count, col
from decimal import Decimal
from pyspark.sql.types import DecimalType, StructType, StructField
import pyspark.sql.functions

In [ ]:
df = df2.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df2.columns]).show()

Replaces null values with value 0

In [ ]:
df2 = df2.fillna('0')

In [ ]:
df = df2.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df2.columns]).show()

Changing the data type in numerical columns:

In [ ]:
df2 = df2.withColumn("TractId", df2["TractId"].cast(IntegerType()))
df2 = df2.withColumn("TotalPop", df2["TotalPop"].cast(IntegerType()))
df2 = df2.withColumn("Men", df2["Men"].cast(IntegerType()))
df2 = df2.withColumn("Women", df2["Women"].cast(IntegerType()))
df2 = df2.withColumn("Hispanic", df2["Hispanic"].cast(FloatType()))
df2 = df2.withColumn("White", df2["White"].cast(FloatType()))
df2 = df2.withColumn("Black", df2["Black"].cast(FloatType()))
df2 = df2.withColumn("Native", df2["Native"].cast(FloatType()))
df2 = df2.withColumn("Asian", df2["Asian"].cast(FloatType()))
df2 = df2.withColumn("Pacific", df2["Pacific"].cast(FloatType()))
df2 = df2.withColumn("Poverty", df2["Poverty"].cast(FloatType()))
df2 = df2.withColumn("ChildPoverty", df2["ChildPoverty"].cast(FloatType()))

In [ ]:
df2.describe(['TotalPop','Men','Asian','Black']).show()

Creating and inserting a column with values

In [ ]:
import pyspark.sql.functions as f

In [ ]:
df2 = df2.withColumn("Others", 100 - f.round(sum(df2["Hispanic","White","Black","Native","Asian","Pacific"]),2))

In [ ]:
df2.show(2)

do fopr all variables

In [ ]:
df2.describe(['TotalPop','Asian','Black','Others']).show()

Converting percentage into values with decimal points:

In [ ]:
df2.createOrReplaceTempView("States")

In [ ]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)
df = sqlContext.sql("SELECT TractId,State, County,TotalPop,Men,Women,\
                         floor(round((Hispanic * TotalPop)/100,0)) as Hispanic, \
                         floor(round((White * TotalPop)/100,0)) as White,\
                         floor(round((Black * TotalPop)/100,0)) as Black, \
                         floor(round((Asian * TotalPop)/100,0)) as Asian, \
                         floor(round((Native * TotalPop)/100,0)) as Native, \
                         floor(round((Pacific * TotalPop)/100,0)) as Pacific,\
                         floor(round((Others * TotalPop)/100,0)) as Others,\
                         floor(round((Poverty * TotalPop)/100,0)) as Poverty, \
                         floor(round((ChildPoverty * TotalPop)/100,0)) as ChildPoverty\
                         FROM States")

In [ ]:
df.show(2)

For Visualisation:

In [ ]:
df.createOrReplaceTempView("States_a")

In [ ]:
dfa = sqlContext.sql("SELECT State, County,sum(TotalPop) as TotalPop,sum(Men) as Men,sum(Women) as Women, sum(Hispanic) as Hispanic,\
                         sum(White) as White, sum(Black) as Black, sum(Asian) as Asian, sum(Native) as Native,sum(Pacific) as Pacific,\
                         sum(Others) as Others,sum(Poverty) as Poverty,sum(ChildPoverty) as ChildPoverty\
                         FROM States_a group by State, County")

In [ ]:
dfa.show(2)

Converting dataframes into csv to be use for visualiation and model output analysis

In [ ]:
df.coalesce(1).write.format('csv').save('/home/danims/Documents/census-data1',header='true')

In [ ]:
dfa.coalesce(1).write.format('csv').save('/home/danims/Documents/census-data2',header='true')

Correlation matrix:

In [ ]:
from pyspark.mllib.stat import Statistics
import pandas as pd
 
#df = sqlCtx.read.format('com.databricks.spark.csv').option('header', 'true').option('inferschema', 'true').load('corr_test.csv')
#dfa = datos
col_names = dfa.columns
features = dfa.rdd.map(lambda row: row[0:])
corr_mat=Statistics.corr(features, method="pearson")
corr_df = pd.DataFrame(corr_mat)
corr_df.index, corr_df.columns = col_names, col_names

In [ ]:
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler

# convert to vector column first
vector_col = "corr_features"
assembler = VectorAssembler(inputCols= ['Hispanic','White','Black','Native','Asian','Pacific', 'Others'], outputCol=vector_col)
df_vector = assembler.transform(dfa).select(vector_col)

# get correlation matrix
matrix = Correlation.corr(df_vector, vector_col)

In [ ]:
matrix.show()

In [ ]:
f_sql = spark.sql ( "SELECT State, max(TotalPop), round(avg(TotalPop),2) as Avg , max(TotalPop) \
                      FROM states \
                      GROUP BY State \
                      ORDER BY Count(*) DESC")
df_sql.show()

Linear Regression:useful when you have data in which a predictions about one variable can be made using the knowlegde about another varaible

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

In [ ]:
vectorAssemble = VectorAssembler(inputCols = ['Hispanic','White','Black','Native','Asian','Pacific', 'Others'], outputCol = "features")

In [ ]:
df_reg = vectorAssemble.transform(df)

In [ ]:
lr = LinearRegression(featuresCol = "features", labelCol = "Poverty")

In [ ]:
lrModel = lr.fit(df_reg)

In [ ]:
lrModel.coefficients

In [ ]:
lrModel.intercept

In [ ]:
lrModel.summary.rootMeanSquaredError

In [ ]:
lrModel.summary.r2

In [ ]:
! ls lr1.model

In [ ]:
df.describe(['Hispanic','White','Black','Native','Asian']).show()

In [ ]:
df.describe(['Pacific', 'Others','Poverty']).show()

In [ ]:
import matplotlib.pyplot as plt

figsize=(100, 100)
values = result_pdf['TotalPop']
names = result_pdf['State']
plt.bar(names, values)
plt.ylabel('some numbers')
plt.show()

In [ ]:
! ls

Decision Tree Regression :

In [ ]:
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler

In [ ]:
vectorAsseDTR = VectorAssembler(inputCols = ['Hispanic','White','Black','Native','Asian','Pacific', 'Others'], outputCol = "features")

In [ ]:
df_dtr = vectorAsseDTR.transform(df)

In [ ]:
splits = df_dtr.randomSplit([0.7,0.3])
train_dtr = splits[0]
test_dtr = splits[1]

In [ ]:
df.count()

In [ ]:
train_dtr.count()

In [ ]:
test_dtr.count()

In [ ]:
dtr = DecisionTreeRegressor(featuresCol = "features", labelCol = "Poverty")
dtr_model = dtr.fit(train_dtr)
dtr_pred = dtr_model.transform(test_dtr)
dtr_eval = RegressionEvaluator(labelCol = "Poverty" , predictionCol = "prediction" , metricName = "rmse")
rmse = dtr_eval.evaluate(dtr_pred)
rmse

In [ ]:
dtr_model = dtr.fit(train_dtr)

In [ ]:
dtr_pred = dtr_model.transform(test_dtr)

In [ ]:
dtr_eval = RegressionEvaluator(labelCol = "Poverty" , predictionCol = "prediction" , metricName = "rmse")

In [ ]:
rmse = dtr_eval.evaluate(dtr_pred)

In [ ]:
rmse

Gradient Boosting Regression

In [ ]:
from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(featuresCol = "features" , labelCol = "Poverty")
gbt_model = gbt.fit(train_dtr)
gbt_pred = gbt_model.transform(test_dtr)
gbt_eval = RegressionEvaluator(labelCol = "Poverty", predictionCol = "prediction", metricName = "rmse")
gbt_rmse = gbt_eval.evaluate(gbt_pred)
gbt_rmse

In [ ]:
gbt = GBTRegressor(featuresCol = "features" , labelCol = "Poverty")

In [ ]:
gbt_model = gbt.fit(train_dtr)

In [ ]:
gbt_pred = gbt_model.transform(test_dtr)

In [ ]:
gbt_eval = RegressionEvaluator(labelCol = "Poverty", predictionCol = "prediction", metricName = "rmse")

In [ ]:
gbt_rmse = gbt_eval.evaluate(gbt_pred)

In [ ]:
gbt_rmse